In [1]:
import numpy as np
import pandas as pd
import requests
import re
import datetime

import locale
locale.setlocale(locale.LC_TIME, "es_ES") #

'es_ES'

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
hoy = "18-11-2022"
columns = ['Subject','Start Date', 'Start Time', 'End Date','All Day Event', 'Description','Location','Private']
# 05/30/20
# 10:00 AM

# Funciones WebScraping

In [4]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException


**Teatro Mayor**

https://www.teatromayor.org/es/temporada/2022

In [5]:
def tmayor_info(driver,url):
    driver.get(url)

    horas = int(driver.find_element('xpath','//div[@class="duration"]//div[@class="hrs"]//span[@class="fact h"]').text)
    minutos = int(driver.find_element('xpath','//div[@class="duration"]//div[@class="min"]//span[@class="fact m"]').text)
    dur_horas = horas+minutos/60
    boletas = driver.find_elements('xpath','//div[@class="content"]//div[@class="table"]//div[@class="row"]//div[@class="col"]')
    desc = driver.find_element('xpath','//div[@class="view-content"]').text.strip()
    try:
        precios = [int(re.findall('^\$(\d*).',boletas[i].text)[0])*1000 for i in range(2,6,3)]
        precio_min, precio_max = min(precios), max(precios)
    except:
        precio_min, precio_max = np.nan, np.nan

    lugar = driver.find_element('xpath','//div[@class="location"]//h4[@class="stage"]').text
    

    return dur_horas, precio_min, precio_max, lugar, desc

In [6]:
def webscraping_teatromayor(año):
    driver = webdriver.Safari()
    caracteristicas = {'tipo-text':'//div[@class="views-field views-field-field-event-field-section-name"]',\
    'titulo-text':'//div[@class="views-field views-field-field-event-title"]',
    'fecha-content':'//div[@class="views-field views-field-field-date-1"]//span[@class="field-content"]//span[@class="date-display-single"]',
    'url-href':'//div[@class="views-field views-field-field-event-title"]//div[@class="field-content"]//a'}

    driver.get("https://www.teatromayor.org/es/temporada/{}".format(año))

    teatro_mayor = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            teatro_mayor[key] = pd.Series([x.text.strip() for x in elementos])
        else:
            teatro_mayor[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    i=1
    while len(driver.find_elements('xpath','//li[@class="pager__item"]//a'))>0:
        i=i+1
        driver.get("https://www.teatromayor.org/es/temporada/{}?page={}".format(año,i))
        teatro_mayor_prov = pd.DataFrame()
        for key in caracteristicas.keys():
            elementos = driver.find_elements('xpath',caracteristicas[key])
            if key.split('-')[1]=='text':
                teatro_mayor_prov[key] = pd.Series([x.text.strip() for x in elementos])
            else:
                teatro_mayor_prov[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])
        teatro_mayor = pd.concat([teatro_mayor,teatro_mayor_prov],ignore_index=True)

    
    teatro_mayor['extrainf'] = teatro_mayor['url-href'].apply(lambda url: tmayor_info(driver,url))
    teatro_mayor['duracion-func'] = teatro_mayor['extrainf'].apply(lambda x: x[0])
    teatro_mayor['preciomin-func'] = teatro_mayor['extrainf'].apply(lambda x: x[1])
    teatro_mayor['preciomax-func'] = teatro_mayor['extrainf'].apply(lambda x: x[2])
    teatro_mayor['lugar-func'] = teatro_mayor['extrainf'].apply(lambda x: x[3])
    teatro_mayor['desc-func'] = teatro_mayor['extrainf'].apply(lambda x: x[4])
    teatro_mayor.drop(columns='extrainf',inplace=True)

    #Limpieza
    teatro_mayor['fecha-content'] = teatro_mayor['fecha-content'].apply(lambda f: f.replace("T"," ").replace(":00-05:00",""))
    driver.quit()

    teatro_mayor.columns = teatro_mayor.columns.str.split("-").str.get(0)
    return teatro_mayor

**Atrapalo**

https://www.atrapalo.com.co/entradas/teatro-y-danza/

In [7]:
def atrapalo_info(driver, url):
    driver.get(url)

    fechas = [fecha_raw.get_attribute('data-date').strip()+" "+hora_raw.text.strip().replace("h","") \
        for fecha_raw,hora_raw in zip(driver.find_elements('xpath','//div[@class="row js-session-group clearl"]'),\
            driver.find_elements('xpath','//span[@class="time-session"]'))]
    
    desc = driver.find_element('xpath','//div[@class="detail-info-box__description container-descripcion app-container-description"]').text.strip()

    return fechas, desc

In [8]:
def webscraping_atrapalo():
    driver = webdriver.Safari()
    driver.get("https://www.atrapalo.com.co/entradas/teatro-y-danza/")

    caracteristicas = {'tipo-text':'//div[@class="item-data event-data"]//p[@class="info"]//span[@class="type large-loc"]',\
    'titulo-title':'//div[@class="item-data event-data"]//h2[@class="clear nombre"]//a',
    'url-href':'//div[@class="item-data event-data"]//h2[@class="clear nombre"]//a',
    'lugar-title':'//div[@class="item-data event-data"]//p[@class="info"]//span//a[@class="locality GATrackEvent_ubicacion hide-for-small-only"]',
    'duracion-text':'//div[@class="event-info-container"]//p[@class="info"][2]',
    'preciomin-text':'//div[@class="price-room "]//span[@class="value "]'}

    atrapalo = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            atrapalo[key] = pd.Series([x.text.strip() for x in elementos])
        else:
            atrapalo[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    dejar_lugares = ['Elteatro.co','Casa E Borrero, Sala Arlequín','Teatro La Mama','Auditorio Sonia Fajardo Forero']

    atrapalo = atrapalo[atrapalo['lugar-title'].isin(dejar_lugares)].reset_index(drop=True)

    # Limpieza
    regex_hora = '(\d+) hora[s]?|(\d+)[ ]?[H,h]'
    regex_minuto = '(\d+)[ ]?[min,Min,MIN][a-z]*|\:(\d+)'
    horas =atrapalo['duracion-text'].apply(lambda durt: int([h[0] if h[0]!='' else h[1] for h in re.findall(regex_hora,str(durt))][0]) if len([h[0] if h[0]!='' else h[1] for h in re.findall(regex_hora,str(durt))])>0 else 0)
    minutos =atrapalo['duracion-text'].apply(lambda durt: int([h[0] if h[0]!='' else h[1] for h in re.findall(regex_minuto,str(durt))][0]) if len([h[0] if h[0]!='' else h[1] for h in re.findall(regex_minuto,str(durt))])>0 else 0)
    atrapalo['duracion-text'] = horas+minutos/60
    atrapalo['preciomin-text']=atrapalo['preciomin-text'].apply(lambda pret: int(re.findall('\$ (\d+).',pret)[0])*1000 if len(re.findall('\$ (\d+).',pret))>0 else np.nan)

    extrainfo = atrapalo['url-href'].apply(lambda url: atrapalo_info(driver,url))
    atrapalo['fecha-func'] = extrainfo.apply(lambda t: t[0])
    atrapalo['desc-func'] = extrainfo.apply(lambda t: t[1])
    atrapalo = atrapalo.explode('fecha-func',ignore_index=True)

    driver.quit()
    atrapalo.columns = atrapalo.columns.str.split("-").str.get(0)
    return atrapalo
    

**Teatro Nacional**

https://teatronacional.co/

In [9]:
def tnacional_info(driver, url):
    driver.get(url)

    titulo = driver.find_elements('xpath','//rs-layer')[-2].text.split(' - ')[0].capitalize()
    lugar = driver.find_elements('xpath','//h3[@class="tbk__title"]')[0].text.split(' - ')[0]
    preciomin = np.nan
    desc = driver.find_elements('xpath','//rs-layer')[-3].text.strip()

    #desc = ""
    # for s in driver.find_elements('xpath','//div[contains(@class,"zn_text_box-light element-scheme--light")]//span[contains(@style,"font-family: Lato;")]'):
    # desc = desc+"\n"+s.text.strip()


    fechas = []

    if len(driver.find_elements('xpath','//button[@class="tc_seating_map_button"]'))>0:
        for k in driver.find_elements('xpath','//button[@class="tc_seating_map_button"]'):
            dtfrm = re.findall('(\d{2} \D{3})',k.text)[0]+' 2022 '+re.findall('(\d.{4,5}M)',k.text)[0].replace('O','0')
            fechas.append(datetime.datetime.strptime(dtfrm,'%d %b %Y %I:%M%p').strftime('%Y-%m-%d %H:%M'))
    else:
        for k in driver.find_elements('xpath','//div[@class="tickera"]//table[@class="event_tickets tickera"]//tbody//tr[last()][td]'):
            dtfrm = re.findall('(\d{2} \D{3})',k.text)[0]+' 2022 '+re.findall('(\d.{4,5}m)',k.text)[0]
            fechas.append(datetime.datetime.strptime(dtfrm,'%d %b %Y %I:%M%p').strftime('%Y-%m-%d %H:%M'))
        
        try:
            preciomin = re.findall('\$(\d+)',\
            driver.find_elements('xpath','//div[@class="tickera"]//table[@class="event_tickets tickera"]//tbody//tr[last()][td]')[-1].text)[0]
        except:
            pass

    return titulo, lugar, fechas, preciomin, desc
    

In [10]:
def webscraping_teatronacional():
    driver = webdriver.Safari()
    driver.get("https://teatronacional.co/")

    caracteristicas = {'url-href':'//a[starts-with(@class,"image-boxes")][not(@href="https://teatronacional.co/")]'}

    teatro_nacional = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            teatro_nacional[key] = pd.Series([x.text.strip() for x in elementos])
        else:
            teatro_nacional[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])


    teatro_nacional_info = teatro_nacional['url-href'].apply(lambda f: tnacional_info(driver, f))
    teatro_nacional["titulo-func"] = teatro_nacional_info.apply(lambda f: f[0])
    teatro_nacional["lugar-func"] = teatro_nacional_info.apply(lambda f: f[1])
    teatro_nacional["fecha-func"] = teatro_nacional_info.apply(lambda f: f[2])
    teatro_nacional["preciomin-func"] = teatro_nacional_info.apply(lambda f: f[3])
    teatro_nacional["desc-func"] = teatro_nacional_info.apply(lambda f: f[4])

    teatro_nacional = teatro_nacional.explode('fecha-func',ignore_index=True)

    driver.quit()

    teatro_nacional.columns = teatro_nacional.columns.str.split("-").str.get(0)
    
    return teatro_nacional

**La Maldita Vanidad**

https://www.lamalditavanidadteatro.com

In [11]:
def webscraping_malditavanidad():
    driver = webdriver.Safari()
    driver.get("https://www.lamalditavanidadteatro.com/")
    iframe = driver.find_element('xpath','//body/div/div/div[3]/div/main/div/div/div/div[2]/div/div/div/wix-iframe/iframe')
    driver.switch_to.frame(iframe)
    driver.implicitly_wait(10)

    items = driver.find_elements('xpath','//div[@class="item"][position() < last()]//h3')

    n = len(items)

    malditavanidad = pd.DataFrame(columns=['titulo','lugar','url','fecha','preciomin','preciomax','duracion','tipo','desc'])

    for i in range(n):
        # Cada vez que entra, accede al iFrame de obras
        driver.get("https://www.lamalditavanidadteatro.com/")
        iframe = driver.find_element('xpath','//body/div/div/div[3]/div/main/div/div/div/div[2]/div/div/div/wix-iframe/iframe')
        driver.switch_to.frame(iframe)
        driver.implicitly_wait(10)
        
        # Hace click en cierta obra y guarda la URL
        item = driver.find_elements('xpath','//div[@class="item"][position() < last()]//h3')[i]
        driver.execute_script("arguments[0].click();",item)
        url = driver.current_url

        # Una vez en la URL busca y pega todos los elementos en p
        driver.get(url)
        titulo = driver.find_elements('xpath','//h2')[1].text.title()
        texto = pd.Series(driver.find_elements('xpath','//div[@data-testid="mesh-container-content"]//div//p')).apply(lambda p: p.text)
        texto = " ".join(texto)

        # Busca las partes del texto que tienen la palabra FECHA
        fecha = re.findall('FECHA.*?:(.*?):',texto)[0]
        # Busca las partes del texto que empiezan por la palabra HORA
        hora = re.findall('HORA.*?:(.*?):',texto)

        # Busca la parte del texto que empieza por la palabra DURACION
        duracion_raw = re.findall('DURACIÓN.*?:(.*?):',texto)[0].strip()
        # Busca si tiene hora
        duracion_hr = re.findall('(\d+) h',duracion_raw)
        # Busca si tiene minuto
        duracion_min = re.findall('(\d+) m',duracion_raw)


        
        # Calcula la duración
        duracion = 0
        if len(duracion_hr)>0:
            duracion += int(duracion_hr[0])
        
        if len(duracion_min)>0:
            duracion += int(duracion_min[0])/60

        # Busca strings de la forma (1-2 digitos) de al? (1-2 digitos) de (mes)
        # 6 de octubre al 9 de octubre
        # Del 13 de octubre al 23 de octubre de 2022
        #Del 27 de octubre  20 de noviembre
        rango_fecha = re.findall('(\d{1,2}) de (\D+) [al ]*(\d{1,2}) de (\w+)',fecha.replace('\xa0',''))
        
        if len(rango_fecha)>0: # Si hay un rango de fechas 
            rango_fecha = rango_fecha[0]
            start_date_raw = str(rango_fecha[0])+" "+rango_fecha[1].split(" al")[0]+" "+str(datetime.datetime.today().year)
            end_date_raw = str(rango_fecha[2])+" "+rango_fecha[3]+" "+str(datetime.datetime.today().year)


            start_date = datetime.datetime.strptime(start_date_raw,'%d %B %Y').strftime('%Y-%m-%d')
            end_date = datetime.datetime.strptime(end_date_raw,'%d %B %Y').strftime('%Y-%m-%d')

            # Por ahora asumiremos que para todas las obras se repiten Jueves,Viernes,Sabado 8 pm y 6:30pm Domingo
            fechas1 = pd.Series(pd.date_range(start=start_date,end=end_date))
            fechas1 = fechas1[fechas1.apply(lambda f: (f.weekday()==3) | (f.weekday()==4) | (f.weekday()==5))] # Deja solo los jueves, viernes o sabados
            fechas1 = fechas1.apply(lambda f: f.replace(hour=20,minute=0)) # Hora son las 8pm
            fechas1 = fechas1.apply(lambda f: f.strftime('%Y-%m-%d %H:%M'))

            fechas2 = pd.Series(pd.date_range(start=start_date,end=end_date))
            fechas2 = fechas2[fechas2.apply(lambda f: f.weekday()==6)] # Deja solo los domingos
            fechas2 = fechas2.apply(lambda f: f.replace(hour=18,minute=30)) # Hora son las 6:30pm
            fechas2 = fechas2.apply(lambda f: f.strftime('%Y-%m-%d %H:%M'))

            fechas_total = pd.concat([fechas1,fechas2]).to_list()
        else:#si la fecha es por 'y' como por ejemplo: Lunes 24 y martes 25 de Octubre de 2022
            fechas = re.findall('(\d{1,2}) y \w* (\d{1,2}) de (\w+)',fecha)[0]
            
            date1_raw = fechas[0]+" "+fechas[2]+" "+str(datetime.datetime.today().year)
            date2_raw = fechas[1]+" "+fechas[2]+" "+str(datetime.datetime.today().year)

            date1 = datetime.datetime.strptime(date1_raw,'%d %B %Y').strftime('%Y-%m-%d')
            date2 = datetime.datetime.strptime(date2_raw,'%d %B %Y').strftime('%Y-%m-%d')

            # Asumimos que las horas son 8pm
            if len(hora)==0:
                date1=pd.Timestamp(date1).replace(hour=20,minute=0)
                date2=pd.Timestamp(date2).replace(hour=20,minute=0)

            fechas_total = [date1,date2]


        # malditavanidad = pd.DataFrame(columns=['titulo','lugar','url','fechas','preciomin','preciomax','duración','tipo'])
        genero = re.findall('GÉNERO.*?:(.*?):',texto)[0].strip().split('RESTRICCIÓN DE EDAD')[0].strip().replace('.','')
        precio = int(re.findall('ENTRADAS.*?\$(\S*)',texto)[0].replace('.',''))
        #Busca la sinopsis del texto
        desc = "".join(re.findall('SINOPSIS.*?:(.+) EL GRUPO|SINOPSIS.*?:(.+) ¿QUÉ|SINOPSIS.*?:(.+)',texto)[0])

        malditavanidad.loc[i] = [titulo,'La Maldita Vanidad',url,fechas_total,precio,np.nan,duracion,genero,desc]

    malditavanidad = malditavanidad.explode(column='fecha',ignore_index=True).dropna(subset='fecha')

    driver.quit()

    return malditavanidad

**Teatro La Candelaria**

https://teatrolacandelaria.com/programacion/

In [12]:
def tcandelaria_info(driver, url):
    #print(url)
    driver.get(url)

    try:
        precio_raw = driver.find_element('xpath','//dd[@class="mec-events-event-cost"]').text.replace('.','')
        precios = [int(x) for x in re.findall('(\d+)',precio_raw)]
        precio_min, precio_max = min(precios), max(precios)
        if precio_min==precio_max:
            precio_max = np.nan
    except NoSuchElementException:
        precio_min = np.nan
        precio_max = np.nan
    

    hora_duration_raw = driver.find_element('xpath','//div[@class="mec-single-event-time"]//dd//abbr').text
    hora_raw = hora_duration_raw.split(' - ')[0]

    try:
        hora_fin =hora_duration_raw.split(' - ')[1]
        duracion = datetime.datetime.strptime(hora_fin,'%I:%M %p')-datetime.datetime.strptime(hora_raw,'%I:%M %p')
        duracion = duracion.seconds/3600
    except:
        duracion = np.nan


    n = len(driver.find_elements('xpath','//div[@class="mec-ticket-available-spots "]//span[@class="mec-event-ticket-name"]'))
    fechas = []

    for i in range(n):
        try:
            fecha_raw = driver.find_elements('xpath','//div[@class="mec-ticket-available-spots "]//span[@class="mec-event-ticket-name"]')[i].text.lower()
            fecha_grupo = re.findall('(\d{1,2}) de (\w+)',fecha_raw)[0]
            dtfrm = fecha_grupo[0]+" "+fecha_grupo[1]+" 2022 "+hora_raw
            fecha = datetime.datetime.strptime(dtfrm,'%d %B %Y %I:%M %p').strftime('%Y-%m-%d %H:%M')
        except: # Coger siguiente y luego restarle una
            fecha_raw = driver.find_elements('xpath','//div[@class="mec-ticket-available-spots "]//span[@class="mec-event-ticket-name"]')[i+1].text.lower()
            fecha_grupo = re.findall('(\d{1,2}) de (\w+)',fecha_raw)[0]
            anterior = int(fecha_grupo[0])-1
            dtfrm = str(anterior)+" "+fecha_grupo[1]+" 2022 "+hora_raw
            fecha = datetime.datetime.strptime(dtfrm,'%d %B %Y %I:%M %p').strftime('%Y-%m-%d %H:%M')
        fechas.append(fecha)

    desc = driver.find_element('xpath','//div[@class="wpb_wrapper"]').text

    return precio_min, precio_max, duracion, fechas, desc

In [13]:
def webscraping_teatrocandelaria():
    driver = webdriver.Safari()
    driver.get("https://teatrolacandelaria.com/programacion/")

    caracteristicas = {'titulo-text':'//a[@class="mec-color-hover"]',\
    'url-href':'//a[@class="mec-color-hover"]'}

    teatro_candelaria = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            teatro_candelaria[key] = pd.Series([x.text.strip().title() for x in elementos])
        else:
            teatro_candelaria[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    #print(teatro_candelaria)
    candelaria_info = teatro_candelaria['url-href'].apply(lambda u: tcandelaria_info(driver,u))
    teatro_candelaria['preciomin-func'] = candelaria_info.apply(lambda f: f[0])
    teatro_candelaria['preciomax-func'] = candelaria_info.apply(lambda f: f[1])
    teatro_candelaria['duracion-func'] = candelaria_info.apply(lambda f: f[2])
    teatro_candelaria['fecha-func'] = candelaria_info.apply(lambda f: f[3])
    teatro_candelaria['desc-func'] = candelaria_info.apply(lambda f: f[4])
    teatro_candelaria = teatro_candelaria.explode('fecha-func',ignore_index=True)
    teatro_candelaria['lugar-func'] = 'Teatro La Candelaria'

    driver.quit()

    teatro_candelaria.columns = teatro_candelaria.columns.str.split("-").str.get(0)

    return teatro_candelaria



**TuBoleta (deprecated)**

https://www.tuboleta.com/eventos/EWvmehRexc8Spkf2vD09GJJgWATkRBUSHOfa56P3GkVPy2evQF1q5e82SPPnucQhDnWECkber20UKvj-+1ovjw==

In [14]:
def tuboleta_info(driver, url): #Deprecated
    driver.get(url)

    if "selection" in url:
        driver.implicitly_wait(20)

        try:
            # Evento unico (Ej: Sala Gaitán)
            
            
            try:
                dia = re.findall('(\d{1,2} \S+ \d{4})',driver.find_element('xpath','//span[@class="unique"]//span[@class="day"]').text.strip())[0]
                hora = driver.find_element('xpath','//span[@class="unique"]//span[@class="time"]').text.strip()

                fechas = [datetime.datetime.strptime(dia+" "+hora,'%d %B %Y %H:%M').strftime('%Y-%m-%d %H:%M')]
                
            except:
                fechas = []

                for k in driver.find_elements('xpath','//div[@class="date_time_venue"]//p[@class="semantic-no-styling-no-display"]'):
                    dtrfm = re.findall('(\d{1,2} \D{3})',k.text)[0]+" 2022 "+re.findall('(\d{1,2}:\d{2})',k.text)[0]
                    fechas.append(datetime.datetime.strptime(dtrfm,'%d %b %Y %H:%M').strftime('%Y-%m-%d %H:%M'))
                


            try:
                preciomin = int(driver.find_elements('xpath','//td[@class="unit_price"]//span[@class="amount amount_COP "]')[-1].get_attribute("data-amount"))/1000
                preciomax = int(driver.find_elements('xpath','//td[@class="unit_price"]//span[@class="amount amount_COP "]')[0].get_attribute("data-amount"))/1000
            except:
                preciomin = int(driver.find_element('xpath','//span[@class="from"]//span[@class="amount amount_COP "]').get_attribute("data-amount"))/1000

                if len(driver.find_elements('xpath','//span[@class="to"]'))>1:
                    preciomax = int(driver.find_element('xpath','//span[@class="to"]//span[@class="amount amount_COP "]').get_attribute("data-amount"))/1000
                else:
                    preciomax=np.nan


            if preciomin==preciomax:
                preciomax=np.nan
        except:
            
            fechas = []
            preciomin = np.nan
            preciomax = np.nan
    else:
        # Evento multiple tipo concierto (Ej: Chavela)
        redirect = driver.find_element('xpath','//div[@class="px-2 text-center py-1 col col-12"]//a').get_attribute('href')

        fechas, preciomin, preciomax = tuboleta_info(driver, redirect)

    return fechas, preciomin, preciomax
            

In [15]:
def webscraping_tuboleta(): #Deprecated
    # Carga la página
    driver = webdriver.Safari()
    driver.get("https://www.tuboleta.com/eventos/EWvmehRexc8Spkf2vD09GJJgWATkRBUSHOfa56P3GkVPy2evQF1q5e82SPPnucQhDnWECkber20UKvj-+1ovjw==")
    driver.implicitly_wait(10)

    # Carga más eventos repetidamente hasta que ya no hay más
    n_old = 0
    n_new = 1

    while n_old<n_new:
        boton = driver.find_element('xpath','//button[@class="ma-2 white--text v-btn v-btn--depressed v-btn--flat v-btn--outlined theme--light v-size--large"]')
        driver.execute_script("arguments[0].click();",boton) 
        driver.implicitly_wait(10)
        n_old = len(driver.find_elements('xpath','//h3[@class="text-center px-3 py-3"]'))

        boton = driver.find_element('xpath','//button[@class="ma-2 white--text v-btn v-btn--depressed v-btn--flat v-btn--outlined theme--light v-size--large"]')
        driver.execute_script("arguments[0].click();",boton) 
        driver.implicitly_wait(10)
        n_new = len(driver.find_elements('xpath','//h3[@class="text-center px-3 py-3"]'))

    caracteristicas = {'titulo-text':'//h3[@class="text-center px-3 py-3"]','lugar-text':'//div[@class="title-place pb-2"]',\
    'url-href':'//div[@class="row px-3"]//div[@class="col col-8"]//div//a'}

    tuboleta = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            tuboleta[key] = pd.Series([x.text.strip() for x in elementos])
        else:
            tuboleta[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    dejar_lugares = ['Sala Gaitán','Teatro Petra','Teatro Jorge Eliécer Gaitán','Teatro Cafam','Teatro Colón Bogotá']

    tuboleta = tuboleta[tuboleta['lugar-text'].isin(dejar_lugares)].reset_index(drop=True)

    tuboletainfo = tuboleta['url-href'].apply(lambda u: tuboleta_info(driver,u))

    tuboleta['fecha-func'] = tuboletainfo.apply(lambda t: t[0])
    tuboleta['preciomin-func'] = tuboletainfo.apply(lambda t: t[1])
    tuboleta['preciomax-func'] = tuboletainfo.apply(lambda t: t[2])
    tuboleta['titulo'] = tuboleta['titulo'].apply(lambda s: s.split(' - ')[0].capitalize())

    driver.quit()

    tuboleta.columns = tuboleta.columns.str.split("-").str.get(0)

    tuboleta = tuboleta.explode(column='fecha',ignore_index=True).dropna(subset='fecha')

    
    return tuboleta

In [16]:
def tuboleta_programacion_info(driver, url):
    driver.get(url)
    
    unicafecha = "unique" in driver.find_element('xpath','//p[@class="semantic-no-styling-no-display date"]//span').get_attribute('class')

    desc = driver.find_element('xpath','//div[@class="description"]').text.strip()
    if not unicafecha: # Varias fechas
        # Caso Múltiples Fechas
        fechas = []

        for k in driver.find_elements('xpath','//div[@class="date_time_venue"]//p[@class="semantic-no-styling-no-display"]'):
            dtrfm = re.findall('(\d{1,2} \D{3})',k.text)[0]+" 2022 "+re.findall('(\d{1,2}:\d{2})',k.text)[0]
            fechas.append(datetime.datetime.strptime(dtrfm,'%d %b %Y %H:%M').strftime('%Y-%m-%d %H:%M'))

        

        #dt1 = datetime.datetime.now()
        #lista_precios = driver.find_elements('xpath','//td[@class="unit_price"]//span[@class="amount amount_COP "]')

        preciomin = int(driver.find_element('xpath','//span[@class="from"]//span[@class="amount amount_COP "]').get_attribute("data-amount"))/1000
    
        if len(driver.find_elements('xpath','//span[@class="to"]'))>1:
            preciomax = int(driver.find_element('xpath','//span[@class="to"]//span[@class="amount amount_COP "]').get_attribute("data-amount"))/1000
        else:
            preciomax=np.nan
        #dt2 = datetime.datetime.now()
        
        #print("Metodo1",dt2-dt1)
        #if len(lista_precios)>0:
        #    preciomin = int(lista_precios[-1].get_attribute("data-amount"))/1000
        #    preciomax = int(lista_precios[0].get_attribute("data-amount"))/1000
        #else:
        #    preciomin = int(driver.find_element('xpath','//span[@class="from"]//span[@class="amount amount_COP "]').get_attribute("data-amount"))/1000
    
        #    if len(driver.find_elements('xpath','//span[@class="to"]'))>1:
        #        preciomax = int(driver.find_element('xpath','//span[@class="to"]//span[@class="amount amount_COP "]').get_attribute("data-amount"))/1000
        #    else:
        #        preciomax=np.nan
            
    


        if preciomin==preciomax:
            preciomax=np.nan

       
    else:
        # Caso Única Fecha

        
        hora = driver.find_element('xpath','//span[@class="time"]').text.strip()

        fecha_raw = driver.find_element('xpath','//span[@class="day"]').text.strip()
        fecha_grupo = re.findall('(\d{1,2}) (\w+) (\d{4})',fecha_raw)[0]

        dtrfm = fecha_grupo[0]+" "+fecha_grupo[1]+" "+fecha_grupo[2]+" "+hora
        fechas = [datetime.datetime.strptime(dtrfm,'%d %B %Y %H:%M').strftime('%Y-%m-%d %H:%M')]

        dt1 = datetime.datetime.now()
        
        
        if len(driver.find_elements('xpath','//li[@class="content_title seat_auto"]//a'))>0:
            driver.get(driver.find_elements('xpath','//li[@class="content_title seat_auto"]//a')[0].get_attribute('href'))
            driver.implicitly_wait(50)
        
        try:
            preciomin = int(driver.find_elements('xpath','//td[@class="unit_price"]//span[@class="amount amount_COP "]')[-1].get_attribute('data-amount'))/1000
            preciomax = int(driver.find_elements('xpath','//td[@class="unit_price"]//span[@class="amount amount_COP "]')[0].get_attribute('data-amount'))/1000

            if preciomax==preciomin:
                preciomax=np.nan
        except:
            preciomin = np.nan
            preciomax = np.nan
        
        dt2 = datetime.datetime.now()
        print(url)
        print(dt2-dt1)

    
    return fechas, preciomin, preciomax
    

**IDARTES (TuBoleta)**

https://www.tuboleta.com/images/Eventos/IDARTES/jorge-eliecer.html

In [17]:
def webscraping_idartes():
    driver = webdriver.Safari()

    teatros_idartes = {"Teatro El Ensueño":"https://www.tuboleta.com/images/Eventos/IDARTES/ensueno.html",\
    "Teatro Jorge Eliecer Gaitán":"https://www.tuboleta.com/images/Eventos/IDARTES/jorge-eliecer.html",\
        "Teatro Jorge Eliecer Gaitán - Sala Gaitán":"https://www.tuboleta.com/images/Eventos/IDARTES/sala-gaitan.html"}

    idartes = pd.DataFrame()
    for teatro in teatros_idartes:
        url_teatro = teatros_idartes[teatro]
        driver.get(url_teatro)
        driver.implicitly_wait(30)
        caracteristicas = {'titulo-text':'//a[starts-with(@class,"sc-")]',\
            'url-href':'//a[starts-with(@class,"sc-")]'}

        temp = pd.DataFrame()
        for key in caracteristicas.keys():
            elementos = driver.find_elements('xpath',caracteristicas[key])
            if key.split('-')[1]=='text':
                temp[key] = pd.Series([x.text.strip().title() for x in elementos])
            else:
                temp[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

        temp = temp[temp['titulo-text']!="Comprar"]
        temp['url-href'] = temp['url-href'].apply(lambda u: u+"&lang=es")
        extrainfo = temp['url-href'].apply(lambda u: tuboleta_programacion_info(driver,u))
        temp['fecha-func'] = extrainfo.apply(lambda f: f[0])
        temp['preciomin-func'] = extrainfo.apply(lambda f: f[1])
        temp['preciomax-func'] = extrainfo.apply(lambda f: f[2])
        temp['lugar-given'] = teatro
        idartes = pd.concat([idartes,temp],ignore_index=True)

    idartes = idartes.explode('fecha-func',ignore_index=True)

    driver.quit()

    idartes.columns = idartes.columns.str.split("-").str.get(0)
    
    return idartes    

**Teatro Colón (TuBoleta)**

https://www.tuboleta.com/images/Eventos/Teatro_Colon/index.html

In [18]:
def webscraping_colon():
    driver = webdriver.Safari()
    driver.get("https://www.tuboleta.com/images/Eventos/Teatro_Colon/index.html")
    driver.implicitly_wait(800)
    tipos = []

    tipos_raw = driver.find_elements('xpath','//div[contains(@class,"stx-Section")]//h2[starts-with(@class,"stx-SectionHeading")]')
    
    for t in tipos_raw:
        xpath = '//div[contains(@class,"'+t.get_attribute('class').split(' sc-')[0].split('SectionHeading-')[-1]+'")]//div//div//div//div//div//a'
        n = len(driver.find_elements('xpath',xpath))
        tipos.extend([t.text.title().strip()]*int(n/2))
        
    xpath = '//div[contains(@class,"stx-SectionHeading stx-SectionHeading")]//div//div//div//div//div//a'

    caracteristicas = {'titulo-text':'//div[contains(@class,"stx-ProductList-EVENTS")]//div//div//div//div//div//div//a',\
        'url-href':'//div[contains(@class,"stx-ProductList-EVENTS")]//div//div//div//div//div//div//a'}

    colon = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            colon[key] = pd.Series([x.text.strip().title() for x in elementos])
        else:
            colon[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])
    
    #colon['tipo-given'] = t.text.strip().title()
    colon = colon[colon['titulo-text']!="Comprar"]
    colon['url-href'] = colon['url-href'].apply(lambda u: u+"&lang=es")
    
    extrainfo = colon['url-href'].apply(lambda u: tuboleta_programacion_info(driver,u))
    colon['fecha-func'] = extrainfo.apply(lambda f: f[0])
    colon['preciomin-func'] = extrainfo.apply(lambda f: f[1])
    colon['preciomax-func'] = extrainfo.apply(lambda f: f[2])

    colon['tipo-given'] = tipos
    colon.loc[:,'lugar-given'] = 'Teatro Colón'

    driver.quit()
    colon = colon.explode('fecha-func',ignore_index=True)
    colon.columns = colon.columns.str.split("-").str.get(0)

    return colon

**Teatro Libre**

https://teatrolibre.com/programacion/

La página tiene un error porque tiene infinitas entradas de la obra Yo Salvaje!, cuando la arreglen se debe hacer que cargue primero todos los "Ver más"

In [19]:
def tlibre_info(driver, url):
    driver.get(url)

    elementos = driver.find_elements('xpath','//div[@class="mec-event-info-desktop mec-event-meta mec-color-before mec-frontbox"]//div//i')

    if len(elementos)>6: # Si hay casilla de precio
        precio_raw = driver.find_element('xpath','//div//div[@class="mec-event-cost"]//dl//dd').text
        
        precio = precio_raw.replace(',','').split('COP')[-1].strip()
        
        precio = int(precio)
    else:
        precio = np.nan

    desc = driver.find_element('xpath','//div[@class="wpb_wrapper"]').text.strip()
        
    return precio, desc

In [20]:
def webscraping_teatrolibre():
    driver = webdriver.Safari()

    driver.get("https://teatrolibre.com/programacion/")
    driver.implicitly_wait(50)

    caracteristicas = {'titulo-text':'//article//div//div//div//h3//a',\
    'url-href':'//article//div//div//div//h3//a',\
        'fecha-text':'//article//div//div//div//div[@class="mec-date-details"]//span//span',\
            'hora-text':'//article//div//div//div//div[@class="mec-time-details"]//span',\
                'lugar-text':'//article//div//div//div//div[@class="mec-venue-details"]/span'}

    teatro_libre = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            teatro_libre[key] = pd.Series([x.text.title().strip() for x in elementos])
        else:
            teatro_libre[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    teatro_libre['fecha-func'] = teatro_libre.apply(lambda f: datetime.datetime.strptime(f['fecha-text']+' 2022 '+f['hora-text'],'%d %b %Y %I:%M %p').strftime('%Y-%m-%d %H:%M'),axis=1)
    teatro_libre = teatro_libre.drop(columns=['fecha-text','hora-text'])
    extrainfo = teatro_libre['url-href'].apply(lambda u: tlibre_info(driver,u))
    teatro_libre['preciomin-func'] = extrainfo.apply(lambda f: f[0])
    teatro_libre['desc-func'] = extrainfo.apply(lambda f: f[1])

    driver.quit()
    teatro_libre.columns = teatro_libre.columns.str.split("-").str.get(0)

    return teatro_libre

**Teatro Colsubsidio**

https://www.tuboleta.com/images/Eventos/Teatro_Colsubsidio/index.html

**Teatro Cafam**

???

**Teatro Petra**

???

# Correr WebScrapings

In [21]:
calendario = pd.DataFrame()

In [22]:
dt1 = datetime.datetime.now()
teatro_mayor1 = webscraping_teatromayor('2022')
dt2 = datetime.datetime.now()
print("Se añadieron {} fechas del Teatro Mayor. Se demoró {} ms.".format(teatro_mayor1.shape[0],(dt2-dt1).microseconds/1000))

Se añadieron 9 fechas del Teatro Mayor. Se demoró 423.956 ms.


In [23]:
calendario = pd.concat([calendario,teatro_mayor1])

In [24]:
dt2 = datetime.datetime.now()
atrapalo = webscraping_atrapalo()
dt1 = datetime.datetime.now()
print("Se añadieron {} fechas de Atrapalo. Se demoró {} ms.".format(atrapalo.shape[0],(dt1-dt2).microseconds/1000))

Se añadieron 19 fechas de Atrapalo. Se demoró 869.873 ms.


In [25]:
calendario = pd.concat([calendario,atrapalo])

In [26]:
dt1 = datetime.datetime.now()
teatro_mayor2 = webscraping_teatromayor('2023')
dt2 = datetime.datetime.now()
print("Se añadieron {} fechas del Teatro Mayor. Se demoró {} ms.".format(teatro_mayor2.shape[0],(dt2-dt1).microseconds/1000))

Se añadieron 145 fechas del Teatro Mayor. Se demoró 30.264 ms.


In [27]:
calendario = pd.concat([calendario,teatro_mayor2])

In [28]:
dt1 = datetime.datetime.now()
teatro_nacional = webscraping_teatronacional()
dt2 = datetime.datetime.now()
print("Se añadieron {} fechas de Teatro Nacional. Se demoró {} ms.".format(teatro_nacional.shape[0],(dt2-dt1).microseconds/1000))

Se añadieron 45 fechas de Teatro Nacional. Se demoró 277.494 ms.


In [29]:
calendario = pd.concat([calendario,teatro_nacional])

In [30]:
dt2 = datetime.datetime.now()
maldita_vanidad = webscraping_malditavanidad()
dt1 = datetime.datetime.now()
print("Se añadieron {} fechas de La Maldita Vanidad. Se demoró {} ms.".format(maldita_vanidad.shape[0],(dt1-dt2).microseconds/1000))

Se añadieron 31 fechas de La Maldita Vanidad. Se demoró 65.835 ms.


In [31]:
calendario = pd.concat([calendario,maldita_vanidad])

In [32]:
dt2 = datetime.datetime.now()
lacandelaria = webscraping_teatrocandelaria()
dt1 = datetime.datetime.now()
print("Se añadieron {} fechas de La Calendaria. Se demoró {} ms.".format(lacandelaria.shape[0],(dt1-dt2).microseconds/1000))

Se añadieron 8 fechas de La Calendaria. Se demoró 117.691 ms.


In [33]:
calendario = pd.concat([calendario,lacandelaria])

In [34]:
dt2 = datetime.datetime.now()
teatro_libre = webscraping_teatrolibre()
dt1 = datetime.datetime.now()
print("Se añadieron {} fechas de Teatro Libre. Se demoró {} ms.".format(teatro_libre.shape[0],(dt2-dt1).microseconds/1000))

Se añadieron 9 fechas de Teatro Libre. Se demoró 434.137 ms.


In [35]:
calendario = pd.concat([calendario,teatro_libre])

In [36]:
dt1 = datetime.datetime.now()
teatro_colon = webscraping_colon()
dt2 = datetime.datetime.now()
print("Se añadieron {} fechas de Teatro Colón - TuBoleta. Se demoró {} ms.".format(teatro_colon.shape[0],(dt1-dt2).microseconds/1000))

https://teatrocolon.checkout.tuboleta.com/selection/event/date?productId=10228740672532&gtmStepTracking=true&lang=es
0:00:01.028154
Se añadieron 3 fechas de Teatro Colón - TuBoleta. Se demoró 374.984 ms.


In [37]:
calendario = pd.concat([calendario,teatro_colon])

In [38]:
dt2 = datetime.datetime.now()
idartes = webscraping_idartes()
dt1 = datetime.datetime.now()
print("Se añadieron {} fechas de IDARTES - TuBoleta. Se demoró {} ms.".format(idartes.shape[0],(dt2-dt1).microseconds/1000))

https://idartes.checkout.tuboleta.com/selection/event/seat?perfId=10228791214140&ot=0&lang=es
0:00:01.543489
https://idartes.checkout.tuboleta.com/selection/event/seat?perfId=10228800137403&ot=0&lang=es
0:00:00.626435
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228821600391&lang=es
0:00:00.633690
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228763653298&lang=es
0:00:30.057224
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228793436338&lang=es
0:00:30.028658
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228793436340&lang=es
0:00:01.532253
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228821600390&lang=es
0:00:50.018587
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228793436341&lang=es
0:00:30.022911
https://idartes.checkout.tuboleta.com/selection/event/date?productId=10228793436342&lang=es
0:00:30.026936
https://idartes.checkout.tuboleta

In [39]:
calendario = pd.concat([calendario,idartes])

In [ ]:
#calendario = pd.concat([teatro_mayor1,teatro_mayor2,atrapalo,teatro_nacional,maldita_vanidad,lacandelaria,idartes,teatro_colon,teatro_libre],ignore_index=True)

In [41]:
calendario = calendario.reset_index(drop=True)

In [42]:
calendario.to_csv("../Guia-Teatral-Bogota/data/guia_teatral({}).csv".format(hoy),sep=',',index=False)

## Festivales Ocasionales

In [ ]:
def fest_circo():
    driver = webdriver.Safari()
    driver.get("https://bogotateatralycircense.gov.co/festival-teatro-y-circo/programacion")

    caracteristicas = {'fecha-text':'//div[@class="evdate mt-3 d-flex justify-content-center"]',\
    'titulo-text':'//div[@class="evtitleobra"]',
    'lugar-text':'//div[@class="evlug"]',
    'url-href':'//div[@class="no-link"]//a','desc-text':'//div[@class="evinfo col-12 p-2"]'}

    fest_circo = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            fest_circo[key] = pd.Series([x.text.strip() for x in elementos])
        else:
            fest_circo[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    fest_circo.columns = fest_circo.columns.str.split("-").str.get(0)

    fest_circo['fecha'] = fest_circo['fecha'].apply(lambda texto: datetime.datetime.strptime(texto.replace('\n','').replace('.','').replace('-','2022').replace(' de',''),'%d %B %Y %I:%M%p').strftime('%Y-%m-%d %H:%M'))
    fest_circo['tipo'] = 'Festival'
    driver.quit()
    return fest_circo

In [ ]:
def fest_opera():
    driver = webdriver.Safari()
    driver.get("https://filarmonicabogota.gov.co/opera-al-parque-2022-programacion/")
    caracteristicas = {'titulo-text':'//a[@class="mec-color-hover"]',\
    'lugar-text':'//div[@class="mec-event-loc-place"]',
    'fecha-text':'//div[@class="event-grid-t2-head mec-bg-color clearfix"]',
    'url-href':'//a[@class="mec-color-hover"]'}

    fest_opera = pd.DataFrame()
    for key in caracteristicas.keys():
        elementos = driver.find_elements('xpath',caracteristicas[key])
        if key.split('-')[1]=='text':
            fest_opera[key] = pd.Series([x.text.strip() for x in elementos])
        else:
            fest_opera[key] = pd.Series([x.get_attribute(key.split('-')[1]) for x in elementos])

    fest_opera.columns = fest_opera.columns.str.split("-").str.get(0)
    fest_opera['fecha'] = fest_opera['fecha'].apply(lambda prueba: datetime.datetime.strptime(prueba.replace('\n                   ' ,'').split('                                                     ')[0].replace('                    ',' 2022').split(' -')[0],'%d %B %Y %I:%M %p').strftime('%Y-%m-%d %H:%M'))
    fest_opera['tipo'] = 'Festival'
    fest_opera['preciomin'] = 0

    driver.quit()

    return fest_opera
    

In [ ]:
calendario = pd.concat([calendario,fest_opera,fest_circo],ignore_index=True)

In [ ]:
calendario

,tipo,titulo,fecha,url,duracion,preciomin,preciomax,lugar,desc
0,Música,‘El elíxir de amor’. Ópera de Gaetano Donizett...,2022-10-26 20:00,https://www.teatromayor.org/es/evento/musica/e...,2.5,150000.0,190000.0,Teatro Mayor Julio Mario Santo Domingo,Producción del Teatro Mayor Julio Mario Santo ...
1,Teatro,Malafama Teatro - 'House of Bernarda' - Direct...,2022-10-28 20:00,https://www.teatromayor.org/es/evento/teatro/t...,1.5,30000.0,40000.0,Teatro Estudio,"Tras la muerte de su segundo esposo, Bernarda ..."
2,Música,‘El elíxir de amor’. Ópera de Gaetano Donizett...,2022-10-28 20:00,https://www.teatromayor.org/es/evento/musica/e...,2.5,150000.0,190000.0,Teatro Mayor Julio Mario Santo Domingo,Producción del Teatro Mayor Julio Mario Santo ...
3,Teatro,Malafama Teatro - 'House of Bernarda' - Direct...,2022-10-29 20:00,https://www.teatromayor.org/es/evento/teatro/t...,1.5,30000.0,40000.0,Teatro Estudio,"Tras la muerte de su segundo esposo, Bernarda ..."
4,Música,‘El elíxir de amor’. Ópera de Gaetano Donizett...,2022-10-30 17:00,https://www.teatromayor.org/es/evento/musica/e...,2.5,150000.0,190000.0,Teatro Mayor Julio Mario Santo Domingo,Producción del Teatro Mayor Julio Mario Santo ...
...,...,...,...,...,...,...,...,...,...
314,Festival,Perdón me tengo que ir,2022-10-28 19:00,https://bogotateatralycircense.gov.co/festival...,NaN,0.0,NaN,Auditorio Biblioteca Pública Gabriel García Má...,Perdón me tengo que ir\n Grupo: Misi - Un...
315,Festival,Anónimas,2022-10-28 19:00,https://bogotateatralycircense.gov.co/festival...,NaN,0.0,NaN,Teatro Quimera Calle 70A 19-40,Anónimas \n Grupo: El escenario \n D...
316,Festival,Colcha de retratos,2022-10-28 19:30,https://bogotateatralycircense.gov.co/festival...,NaN,0.0,NaN,Casa Teatrova Calle 24 No. 4 -16,Colcha de retratos \n Grupo: Asociación C...
317,Festival,Evento especial: House of Bernarda,2022-10-28 20:00,https://bogotateatralycircense.gov.co/festival...,NaN,0.0,NaN,Teatro estudio - Teatro Mayor Julio Mario Sant...,Evento especial: House of Bernarda\n Grup...


In [ ]:
calendario.to_csv("../Guia-Teatral-Bogota/data/guia_teatral({}).csv".format(hoy),sep=',',index=False)